In [ ]:
import tensorflow as tf
import os
from PIL import Image

def _bytes_feature(value):
    """Returns a bytes_list from a string/byte."""
    if isinstance(value, str):
        value = value.encode('utf-8')
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from an int."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def create_tfrecord_from_images(image_dir, output_path, class_names):
    """
    image_dir: directory containing subfolders for each class
    output_path: path to save the TFRecord file
    class_names: list of class names in sorted order
    """
    with tf.io.TFRecordWriter(output_path) as writer:
        for class_idx, class_name in enumerate(class_names):
            class_folder = os.path.join(image_dir, class_name)
            for image_name in os.listdir(class_folder):
                image_path = os.path.join(class_folder, image_name)
                
                # Load and resize the image
                image = Image.open(image_path).convert('RGB')
                image = image.resize((224, 224))
                image_bytes = image.tobytes()

                # Create a feature dictionary
                feature = {
                    'image': _bytes_feature(image_bytes),
                    'label': _int64_feature(class_idx)
                }

                # Create an Example
                example = tf.train.Example(features=tf.train.Features(feature=feature))
                writer.write(example.SerializeToString())
    print(f"TFRecord created at {output_path}")
